In [1]:
!pip install git+https://github.com/openai/CLIP.git
!pip install kaolin==0.17.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html
!pip install open-clip-torch

!pip install open3d

import os

if not os.path.exists("AML_group"):
    !git clone https://github.com/AlesCarl/AML_group.git
else:
    print("Repository già clonato.")
%cd AML_group




  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-z2ojwsty
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-z2ojwsty
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
Looking in links: https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html
Repository già clonato.
/content/AML_group


In [23]:
!pip install Pillow

In [2]:
!git pull

remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 298 bytes | 298.00 KiB/s, done.
From https://github.com/AlesCarl/AML_group
   754d5c3..2ad18d3  main       -> origin/main
Updating 754d5c3..2ad18d3
Fast-forward
 render.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


# Part1

In [3]:
import clip
import copy
import json
import kaolin as kal
import kaolin.ops.mesh
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torchvision
import open_clip
import open3d as o3d


from itertools import permutations, product
from Normalization import MeshNormalizer


from mesh import Mesh
from pathlib import Path
from render import Renderer
from tqdm import tqdm
from torch.autograd import grad
from torchvision import transforms
from utils import device, color_mesh

class NeuralHighlighter(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(NeuralHighlighter, self).__init__()

        layers = []
        for i in range(num_layers):
            layers.append(nn.Linear(input_dim if i == 0 else hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.LayerNorm([hidden_dim]))

        layers.append(nn.Linear(hidden_dim, output_dim))
        layers.append(nn.Softmax(dim=1))

        self.mlp = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.mlp:
            x = layer(x)
        return x


def get_clip_model(clip_model):
    device = 'cuda'
    model, preprocess = clip.load(clip_model, device=device) # jit = True for better perfomance
    return model



# ================== HELPER FUNCTIONS =============================
def save_final_results(log_dir, name, mesh, mlp, vertices, colors, render, background):
    mlp.eval()
    with torch.no_grad():
        probs = mlp(vertices)
        max_idx = torch.argmax(probs, 1, keepdim=True)
        # for renders
        one_hot = torch.zeros(probs.shape).to(device)
        one_hot = one_hot.scatter_(1, max_idx, 1)
        sampled_mesh = mesh

        highlight = torch.tensor([204, 255, 0]).to(device)
        gray = torch.tensor([180, 180, 180]).to(device)
        colors = torch.stack((highlight/255, gray/255)).to(device)
        color_mesh(one_hot, sampled_mesh, colors)
        rendered_images, _, _ = render.render_views(sampled_mesh, num_views=5,
                                                                        show=False,
                                                                        center_azim=0,
                                                                        center_elev=0,
                                                                        std=1,
                                                                        return_views=True,
                                                                        lighting=True,
                                                                        background=background)
        # for mesh
        final_color = torch.zeros(vertices.shape[0], 3).to(device)
        final_color = torch.where(max_idx==0, highlight, gray)
        mesh.export(os.path.join(log_dir, f"{name}.ply"), extension="ply", color=final_color)
        save_renders(log_dir, 0, rendered_images, name='final_render.jpg')


def clip_loss(n_augs, rendered_images, encoded_text, clip_transform, augment_transform, clip_model):

    if n_augs == 0:
        clip_image = clip_transform(rendered_images)
        encoded_renders = clip_model.encode_image(clip_image)
        encoded_renders = encoded_renders / encoded_renders.norm(dim=1, keepdim=True)
        if encoded_text.shape[0] > 1:
            loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                torch.mean(encoded_text, dim=0), dim=0)
        else:
            loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                encoded_text)
    elif n_augs > 0:
        loss = 0.0
        for _ in range(n_augs):
            augmented_image = augment_transform(rendered_images)
            encoded_renders = clip_model.encode_image(augmented_image)
            if encoded_text.shape[0] > 1:
                loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                    torch.mean(encoded_text, dim=0), dim=0)
            else:
                loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                    encoded_text)

        loss= loss / n_augs

    return loss
    # "1-loss" removed -> now best value in output is -1

def save_renders(dir, i, rendered_images, name=None):
    if name is not None:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, name))
    else:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, 'renders/iter_{}.jpg'.format(i)))


Warp 1.5.1 initialized:
   CUDA Toolkit 12.6, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.5.1


In [21]:
# Constrain most sources of randomness
# (some torch backwards functions within CLIP are non-determinstic)
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


render_res = 224
n_iter = 1000
res = 224
obj_path = 'data/horse.obj' # 'APPROXIMATE.obj' if we want to test the point cloud to mesh conversion
output_dir = './output/'
clip_model = 'ViT-L/14'

input_dim = 3
hidden_dim = 256
output_dim = 2

# Hyper-parameters
learning_rate = 0.0001
n_layers = 3 # depth 4
n_views = 5 # 5
n_augs = 1 # 4

Path(os.path.join(output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

objbase, extension = os.path.splitext(os.path.basename(obj_path))

render = Renderer(dim=(render_res, render_res))
mesh = Mesh(obj_path)
MeshNormalizer(mesh)()


### --

# Initialize variables
background = torch.tensor((1., 1., 1.)).to(device)

log_dir = output_dir


# CLIP and augmentation transform
clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
clip_transform = transforms.Compose([
    transforms.Resize((res, res)),
    clip_normalizer
])
augment_transform = transforms.Compose([
    transforms.RandomResizedCrop(res, scale=(1, 1)),
    transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
    clip_normalizer
])



# MLP Settings
mlp = NeuralHighlighter(input_dim, hidden_dim, output_dim, n_layers).to(device)
optim = torch.optim.Adam(mlp.parameters(), learning_rate)

# list of possible colors
rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
colors = torch.tensor(full_colors).to(device)


# --- Prompt ---
# encode prompt with CLIP
model = get_clip_model(clip_model)

known_object = 'horse'
classes = 'Shoes'


prompt = "A 3D render of a gray {} with highlighted {}".format(known_object, classes)
with torch.no_grad():
    prompt_token = clip.tokenize([prompt]).to(device)
    encoded_text = model.encode_text(prompt_token)
    encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)


vertices = copy.deepcopy(mesh.vertices)

losses = []

# Optimization loop
for i in tqdm(range(n_iter)):
    optim.zero_grad()

    # predict highlight probabilities
    pred_class = mlp(vertices)

    # color and render mesh
    sampled_mesh = mesh
    color_mesh(pred_class, sampled_mesh, colors)
    rendered_images, elev, azim = render.render_views(sampled_mesh, num_views=n_views,
                                                            show=False,
                                                            center_azim=0,
                                                            center_elev=0,
                                                            std=1,
                                                            return_views=True,
                                                            lighting=True,
                                                            background=background)

    # Calculate CLIP Loss
    loss = clip_loss(n_augs, rendered_images, encoded_text, clip_transform, augment_transform, model)
    loss.backward(retain_graph=True)

    optim.step()

    # update variables + record loss
    with torch.no_grad():
        losses.append(loss.item())

    # report results
    if i % 100 == 0:
        print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
        save_renders(log_dir, i, rendered_images)
        with open(os.path.join(log_dir, "training_info.txt"), "a") as f:
            f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")


# save results
save_final_results(log_dir, 'Primo test', mesh, mlp, vertices, colors, render, background)

# Save prompts
with open(os.path.join(dir, prompt), "w") as f:
    f.write('')

NameError: name 'torch' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# part 2

In [ ]:
# Percorso del file OBJ
obj_path = 'data/horse.obj'

# Funzione per caricare la mesh come TriangleMesh
def load_obj_as_triangle_mesh(obj_path):
    # Carica il file .obj
    mesh = o3d.io.read_triangle_mesh(obj_path)
    if mesh.is_empty():
        raise ValueError(f"La mesh nel file {obj_path} non è stata caricata correttamente.")
    return mesh

# Carica la mesh
mesh = load_obj_as_triangle_mesh(obj_path)

# Converte la mesh in una point cloud campionando punti uniformemente
pcd = mesh.sample_points_uniformly(2048)

# Imposta il colore nero per tutti i punti
pcd.colors = o3d.utility.Vector3dVector(np.zeros((len(pcd.points), 3)))  # Colore nero: [0, 0, 0]


# Esporta la point cloud in formato PLY
# o3d.io.write_point_cloud("candle_PC.ply", pcd)

In [ ]:
# Point cloud to mesh

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)

o3d.io.write_triangle_mesh("APPROXIMATE.obj", mesh)

# 3 part

In [ ]:
import os
from os.path import join as opj
import numpy as np
from torch.utils.data import Dataset
import h5py
import json
import pickle as pkl


def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc, centroid, m


class AffordNetDataset(Dataset):
    def __init__(self, data_dir, split):
        super().__init__()
        self.data_dir = data_dir
        self.split = split

        self.load_data()

        self.affordance = self.all_data[0]["affordance"]

        return

    def load_data(self):
     self.all_data = []
     with open(opj(self.data_dir, 'full_shape_%s_data.pkl' % self.split), 'rb') as f:
        temp_data = pkl.load(f)
     for index, info in enumerate(temp_data):
        if info["semantic class"] == "Scissors":  # Filtra solo gli oggetti "BOWL"
            temp_info = {}
            temp_info["shape_id"] = info["shape_id"]
            temp_info["semantic class"] = info["semantic class"]
            temp_info["affordance"] = info["affordance"]
            temp_info["data_info"] = info["full_shape"] # vertici
            self.all_data.append(temp_info)




    def __getitem__(self, index):

        data_dict = self.all_data[index]
        modelid = data_dict["shape_id"]
        modelcat = data_dict["semantic class"]

        data_info = data_dict["data_info"]
        model_data = data_info["coordinate"].astype(np.float32)
        labels = data_info["label"]
        for aff in self.affordance:
            temp = labels[aff].astype(np.float32).reshape(-1, 1)
            model_data = np.concatenate((model_data, temp), axis=1)

        datas = model_data[:, :3]
        targets = model_data[:, 3:]

        datas, _, _ = pc_normalize(datas)

        return datas, datas, targets, modelid, modelcat

    def __len__(self):
        return len(self.all_data)

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Subset

def build_dataset(data_dir, test=False):
    test_set = AffordNetDataset(data_dir, 'train')
    val_set = AffordNetDataset(data_dir, 'val')

    # Seleziona i primi 5 campioni per il VAL set
    val_indices = list(range(min(5, len(val_set))))
    val_set = Subset(val_set, val_indices)

    # Seleziona 10 campioni per il TEST set, a partire dal 6° (indice 5)
    test_start_index = 5
    test_indices = list(range(test_start_index, test_start_index + min(10, len(test_set) - test_start_index)))
    test_set = Subset(test_set, test_indices)

    # Ritorna i dataset in un dizionario
    dataset_dict = dict(val_set=val_set, test_set=test_set)
    return dataset_dict


def build_loader(dataset_dict):
    val_set = dataset_dict["val_set"]
    test_set = dataset_dict["test_set"]

    batch_size_factor = 1


    test_loader = DataLoader(test_set, batch_size=1, ## occhio qui
                              shuffle=True, drop_last=False, num_workers=8)

    val_loader = DataLoader(val_set, batch_size=1,
                            shuffle=False, num_workers=8, drop_last=False)
    loader_dict = dict(
        val_loader=val_loader,
        test_loader=test_loader ,
    )

    return loader_dict

In [ ]:
# dataset_dir = '/content/drive/My Drive/full-shape'
dataset_dir = '/content/drive/My Drive/ColabNotebooks'

dataset = build_dataset(dataset_dir)

loader = build_loader(dataset)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
def pointToMesh(data ):
    #'''
    single_data = data[0].cpu().numpy()     # shape [N, 3]

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(single_data)

    #o3d.io.write_point_cloud("PointCloud.ply", pcd)


    # Stima delle normali
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=0.06,  # Raggio per la stima delle normali,
        max_nn=80     # Numero massimo di vicini per stimare la normale
    ))

    # Orienta le normali in modo consistente
    pcd.orient_normals_consistent_tangent_plane(k=100) # 30

    # Ricostruzione con
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
        pcd, 0.04 #0.05
    )

    # (Opzionale) Liscia la mesh per rimuovere artefatti
    ## mesh = mesh.filter_smooth_laplacian(number_of_iterations=5)

    # Salva e visualizza la mesh
    o3d.io.write_triangle_mesh("alpha.obj", mesh)
    #'''



## FUNZIONE

In [ ]:
def optimize (vertex, targets , modelCat, n_layers, n_views, n_augs, prompt):

    pointToMesh(vertex) #

    seed = 42
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


    render_res = 224
    n_iter = 1000
    res = 224
    obj_path = 'alpha.obj'
    output_dir = './output/'
    clip_model = 'ViT-L/14'

    input_dim = 3
    hidden_dim = 256
    output_dim = 2

    # Hyper-parameters
    learning_rate = 0.0001
    ##
    ##
    ##


    Path(os.path.join(output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

    objbase, extension = os.path.splitext(os.path.basename(obj_path))

    render = Renderer(dim=(render_res, render_res))
    mesh = Mesh(obj_path)
    MeshNormalizer(mesh)()

    # Initialize variables
    background = torch.tensor((1., 1., 1.)).to(device)

    log_dir = output_dir

    # CLIP and augmentation transform
    clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
    clip_transform = transforms.Compose([
        transforms.Resize((res, res)),
        clip_normalizer
    ])
    augment_transform = transforms.Compose([
        transforms.RandomResizedCrop(res, scale=(1, 1)),
        transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
        clip_normalizer
    ])

    # MLP Settings
    mlp = NeuralHighlighter(input_dim, hidden_dim, output_dim, n_layers).to(device)
    optim = torch.optim.Adam(mlp.parameters(), learning_rate)

    # list of possible colors
    rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
    color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
    full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
    colors = torch.tensor(full_colors).to(device)

    # --- Prompt ---
    # encode prompt with CLIP
    model = get_clip_model(clip_model)

    known_object = modelCat






    with torch.no_grad():
        prompt_token = clip.tokenize([prompt]).to(device)
        encoded_text = model.encode_text(prompt_token)
        encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)

    vertices = copy.deepcopy(mesh.vertices)

    losses = []

    # Optimization loop
    for i in tqdm(range(n_iter)):
        optim.zero_grad()

        # predict highlight probabilities
        pred_class = mlp(vertices)

        # color and render mesh
        sampled_mesh = mesh
        color_mesh(pred_class, sampled_mesh, colors)
        rendered_images, elev, azim = render.render_views(sampled_mesh, num_views=n_views,
                                                                show=False,
                                                                center_azim=0,
                                                                center_elev=0,
                                                                std=1,
                                                                return_views=True,
                                                                lighting=True,
                                                                background=background)

        # Calculate CLIP Loss
        loss = clip_loss(n_augs, rendered_images, encoded_text, clip_transform, augment_transform, model)
        loss.backward(retain_graph=True)

        optim.step()

        # update variables + record loss
        with torch.no_grad():
            losses.append(loss.item())

        # report results
        if i % 100 == 0:
            print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
            save_renders(log_dir, i, rendered_images)
            with open(os.path.join(log_dir, "training_info.txt"), "a") as f:
                f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")
        #if i == 700:
        #  break

    # save results
    save_final_results(log_dir, 'final_mesh', mesh, mlp, vertices, colors, render, background)

    return mlp


## Test 2

In [ ]:
def calculate_miou(predictions, ground_truths, threshold):

    print(f"Predictions shape: {predictions.shape}")
    print(f"Ground truths shape: {ground_truths.shape}")

    #predictions = (predictions >= threshold).astype(int)  # Binarizza le predizioni
    ground_truths = (ground_truths >= threshold).astype(int)  # Binarizza ground truth

    batch_size, num_points = ground_truths.shape
    iou_per_class = np.zeros((batch_size, 1))

    for b in range(batch_size):
        pred = predictions[b, :].cpu().numpy()
        gt = ground_truths[b, :]

        # Calcola intersezione e unione
        intersection = np.sum(pred * gt)
        union = np.sum(pred + gt) - intersection

        if union == 0:  # Evita divisione per zero
            iou_per_class[b, 0] = np.nan  # Non valido se non ci sono punti
        else:
            iou_per_class[b, 0] = intersection / union


    # Media su batch e classi
    mean_iou = np.nanmean(iou_per_class)
    return mean_iou

# RUN val- test

In [ ]:
model_instance = None
targets = None
vertex = None

n_layers = 5 # depth 4
n_views = 5 # 4
n_augs = 4 # 4


# attualmente per me 554

# 657 optimal ?


# prompt= " A 3D rendering of a pair of scissors with handles designed for fingers to be inserted"
prompt= " A 3D rendering of scissors showing the regions optimized for grasping" # by hand"




for batch_idx, batch in enumerate(loader["val_loader"]):


    if batch_idx == 0: # MESH buona
       continue


    if batch_idx == 1: # forbice strana
       continue



    if batch_idx == 2: # forbice aperta
       continue

    '''

    if batch_idx == 3:
       continue

    '''


    # Estraggo il primo elemento del batch
    data, data1, targets, modelid, modelcat = batch   # con target = GT  ( area giusta da colorare )
    vertex = data
    pointToMesh(data) #


    model_instance = optimize(data, targets, modelcat, n_layers, n_views, n_augs, prompt)


    break



  0%|          | 0/1000 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 3.06 MiB is free. Process 4704 has 14.74 GiB memory in use. Of the allocated memory 13.16 GiB is allocated by PyTorch, and 1.44 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model_instance_test = None
targets = None
vertex = None


for batch_idx, batch2 in enumerate(loader["test_loader"]):



    if batch_idx == 0: # male male
       continue
    '''

    if batch_idx == 1:
       continue


    if batch_idx == 2: # forbice aperta
       continue



    if batch_idx == 3:
       continue

    '''


    # Estraggo il primo elemento del batch
    data, data1, targets, modelid, modelcat = batch2   # con target = GT  ( area giusta da colorare )
    #print(data)

    vertex = data
    #pointToMesh(data) #


    model_instance_test = optimize(data, targets, modelcat)

    break



In [ ]:
model_instance1= model_instance
model_instance1.eval()  # Metti il modello in modalità eval per la validazione/test

with torch.no_grad():  # Disabilita il calcolo dei gradienti per l'inferenza


  # Predizioni del modello
  vertex, targets = vertex.float().cuda(), targets.float().cuda()

  afford_pred = model_instance1(vertex)
  #afford_pred = torch.sigmoid(afford_pred).detach().cpu()  # Shape: [1, 2048, 2]


  # Usa argmax per ottenere la classe predetta (0 o 1)
  afford_pred = torch.argmax(afford_pred, dim=-1)  # Shape: [1, 2048]

  # Usare l'operazione logica NOT (bitwise XOR con 1)
  afford_pred = afford_pred ^ 1


  new_targets = targets[:, :, 0]

  #for value in afford_pred[0]:
    #print(value)


 #  for value in new_targets[0]:
 #    print(value)

  # Ground truth
  ground_truth = new_targets.detach().cpu().numpy()  # Shape: [1, 2048, 1]



# Calcolo del mIOU
miou = calculate_miou(afford_pred, ground_truth, threshold=0.1)
print(f"Mean IOU: {miou}")

Predictions shape: torch.Size([1, 2048])
Ground truths shape: (1, 2048)
Mean IOU: 0.3295638126009693


In [ ]:

# Numero di oggetti nel validation loader
num_val_objects = len(loader["val_loader"].dataset)
print(f"Number of objects in val_loader: {num_val_objects}")

# Numero di oggetti nel test loader
num_test_objects = len(loader["test_loader"].dataset)
print(f"Number of objects in test_loader: {num_test_objects}")


Number of objects in val_loader: 5
Number of objects in test_loader: 10


# TODO: domani:

-

## vari prompt da provare:

- "A pair of sharp scissors with glowing yellow cutting edges designed for cutting."

- "A 3D render of scissors cutting fabric, where the sharp metal edges are glowing yellow to highlight their cut functionality."


In [ ]:
!rm -rf /content/AML_group


## Extension

In [ ]:
render = Renderer(dim=(224, 224), background_image='data/bg.jpg')
mesh = Mesh('data/horse.obj')
MeshNormalizer(mesh)()
background = torch.tensor((1., 1., 1.)).to(device)

rendered_images, elev, azim = render.render_views(mesh,
                                                  num_views=5,
                                                  show=False,
                                                  center_azim=0,
                                                  center_elev=0,
                                                  std=1,
                                                  return_views=True,
                                                  lighting=True,
                                                  background=background)

save_renders('./', 0, rendered_images)

In [ ]:
from torchvision.transforms import functional as F

class BackgroundTransform:
    def __init__(self, backgrounds, blur_sigma=(0.1, 2.0)):
        """
        Args:
            backgrounds (list of torch.Tensor): List of background images as tensors.
            blur_sigma (tuple): Min and max sigma for Gaussian blur.
        """
        self.backgrounds = backgrounds
        self.blur_sigma = blur_sigma

    def __call__(self, rendered_image):
        # Select a random background
        bg = random.choice(self.backgrounds)

        # Apply Gaussian blur to the background
        blurred_bg = F.gaussian_blur(bg, kernel_size=(5, 5), sigma=random.uniform(*self.blur_sigma))

        # Blend the rendered image with the blurred background
        mask = (rendered_image.sum(dim=0, keepdim=True) > 0).float()  # Mask from rendered image
        transformed_image = rendered_image * mask + blurred_bg * (1 - mask)

        return transformed_image

In [ ]:
from PIL import Image

# Constrain most sources of randomness
# (some torch backwards functions within CLIP are non-determinstic)
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


render_res = 224
n_iter = 1000
res = 224
obj_path = 'data/horse.obj' # 'APPROXIMATE.obj' if we want to test the point cloud to mesh conversion
output_dir = './output/'
clip_model = 'ViT-L/14'

input_dim = 3
hidden_dim = 256
output_dim = 2

# Hyper-parameters
learning_rate = 0.0001
n_layers = 3 # depth 4
n_views = 5 # 5
n_augs = 1 # 4

Path(os.path.join(output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

objbase, extension = os.path.splitext(os.path.basename(obj_path))

render = Renderer(dim=(render_res, render_res))
mesh = Mesh(obj_path)
MeshNormalizer(mesh)()

backgrounds = [
    transforms.ToTensor()(Image.open("data/bg.jpg").resize((res, res))).to(device),
    torch.tensor((1., 1., 1.)).to(device)
]

log_dir = output_dir

# CLIP and augmentation transform
clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
clip_transform = transforms.Compose([
    transforms.Resize((res, res)),
    clip_normalizer
])
augment_transform = transforms.Compose([
    BackgroundTransform(backgrounds, blur_sigma=(0.1, 2.0)),
    transforms.RandomResizedCrop(res, scale=(1, 1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
    transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0)),
    clip_normalizer
])

# MLP Settings
mlp = NeuralHighlighter(input_dim, hidden_dim, output_dim, n_layers).to(device)
optim = torch.optim.Adam(mlp.parameters(), learning_rate)

# list of possible colors
rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
colors = torch.tensor(full_colors).to(device)


# --- Prompt ---
# encode prompt with CLIP
model = get_clip_model(clip_model)

known_object = 'horse'
classes = 'Shoes'


prompt = "A 3D render of a gray {} with highlighted {}".format(known_object, classes)
with torch.no_grad():
    prompt_token = clip.tokenize([prompt]).to(device)
    encoded_text = model.encode_text(prompt_token)
    encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)


vertices = copy.deepcopy(mesh.vertices)

losses = []

# Optimization loop
for i in tqdm(range(n_iter)):
    optim.zero_grad()

    # predict highlight probabilities
    pred_class = mlp(vertices)

    # color and render mesh
    sampled_mesh = mesh
    color_mesh(pred_class, sampled_mesh, colors)
    rendered_images, elev, azim = render.render_views(sampled_mesh, num_views=n_views,
                                                            show=False,
                                                            center_azim=0,
                                                            center_elev=0,
                                                            std=1,
                                                            return_views=True,
                                                            lighting=True,
                                                            background=background)

    # Calculate CLIP Loss
    loss = clip_loss(n_augs, rendered_images, encoded_text, clip_transform, augment_transform, model)
    loss.backward(retain_graph=True)

    optim.step()

    # update variables + record loss
    with torch.no_grad():
        losses.append(loss.item())

    # report results
    if i % 100 == 0:
        print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
        save_renders(log_dir, i, rendered_images)
        with open(os.path.join(log_dir, "training_info.txt"), "a") as f:
            f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")


# save results
save_final_results(log_dir, 'Primo test', mesh, mlp, vertices, colors, render, background)

# Save prompts
with open(os.path.join(dir, prompt), "w") as f:
    f.write('')